<a href="https://colab.research.google.com/github/prog815/learnTensorFlow/blob/master/tf_data_%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B2%D1%85%D0%BE%D0%B4%D0%BD%D1%8B%D1%85_%D0%BA%D0%BE%D0%BD%D0%B2%D0%B5%D0%B9%D0%B5%D1%80%D0%BE%D0%B2_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tf.data: создание входных конвейеров TensorFlow.

https://www.tensorflow.org/guide/data

Енин А.В. 

10.08.2020

Оренбург

In [1]:
import tensorflow as tf

In [2]:
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,4,3,2,1])
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [6]:
for elem in dataset:
    print(elem.numpy())

1
2
3
4
5
4
3
2
1


In [9]:
it = iter(dataset)
print(next(it).numpy())

1


In [12]:
dataset.reduce(0,lambda st,vl : st + vl**2).numpy()

85

In [38]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform((4,3)))
dataset1.element_spec

TensorSpec(shape=(3,), dtype=tf.float32, name=None)

In [21]:
it = iter(dataset1)
it.next()

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.85088086, 0.20761049, 0.9651443 ], dtype=float32)>

In [37]:
dataset2 = tf.data.Dataset.from_tensor_slices((
    tf.random.uniform((4,)),
    tf.random.uniform((4,3),maxval=10,dtype=tf.int32)
))
dataset2.element_spec

(TensorSpec(shape=(), dtype=tf.float32, name=None),
 TensorSpec(shape=(3,), dtype=tf.int32, name=None))

In [36]:
d = dataset2.take(1)
print(d)
d.as_numpy_iterator().next()

<TakeDataset shapes: ((), (3,)), types: (tf.float32, tf.int32)>


(0.46568036, array([7, 5, 4], dtype=int32))

In [51]:
dataset3 = tf.data.Dataset.zip((dataset1,dataset2))
dataset3.element_spec

(TensorSpec(shape=(3,), dtype=tf.float32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(3,), dtype=tf.int32, name=None)))

In [54]:
t = tf.SparseTensor(indices=((0,0),(1,2)),values=(1,2),dense_shape=(3,4))
dataset4 = tf.data.Dataset.from_tensors(t)
dataset4.element_spec

SparseTensorSpec(TensorShape([3, 4]), tf.int32)

In [55]:
dataset4.element_spec.value_type

tensorflow.python.framework.sparse_tensor.SparseTensor

# Преобразования

In [60]:
dataset1 = tf.data.Dataset.from_tensor_slices(
    tf.random.uniform((4,10),minval=1,maxval=10,dtype=tf.int32)
)
dataset1

<TensorSliceDataset shapes: (10,), types: tf.int32>

In [64]:
for z in dataset1:
    print(z)

tf.Tensor([5 9 3 1 7 9 2 5 5 3], shape=(10,), dtype=int32)
tf.Tensor([8 7 3 3 6 4 3 4 9 9], shape=(10,), dtype=int32)
tf.Tensor([8 5 5 3 2 9 7 8 7 9], shape=(10,), dtype=int32)
tf.Tensor([2 3 3 5 9 3 2 4 6 3], shape=(10,), dtype=int32)


In [67]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (
        tf.random.uniform((4,)),
        tf.random.uniform((4,100),maxval=10,dtype=tf.int32)
    )
)
dataset2

<TensorSliceDataset shapes: ((), (100,)), types: (tf.float32, tf.int32)>

In [69]:
dataset3 = tf.data.Dataset.zip((dataset1,dataset2))
dataset3

<ZipDataset shapes: ((10,), ((), (100,))), types: (tf.int32, (tf.float32, tf.int32))>

In [72]:
for a,(b,c) in dataset3:
    print('shapes:',a.shape,b.shape,c.shape)

shapes: (10,) () (100,)
shapes: (10,) () (100,)
shapes: (10,) () (100,)
shapes: (10,) () (100,)


# Чтение входных данных

In [74]:
train,test = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [80]:
train[0].shape,train[1].shape

((60000, 28, 28), (60000,))

In [87]:
images,labels = train
images = images / 255

In [88]:
dataset = tf.data.Dataset.from_tensor_slices((images,labels))
dataset

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

## Использование генераторов

In [89]:
def count(stop):
    i = 0
    while i<stop:
        yield i
        i += 1

In [91]:
for n in count(5):
    print(n)

0
1
2
3
4


In [94]:
ds_counter = tf.data.Dataset.from_generator(count,output_types=tf.int32,output_shapes=(),args=[25])
ds_counter

<FlatMapDataset shapes: (), types: tf.int32>

In [98]:
for count_batch in ds_counter.repeat().batch(10).take(10):
    print(count_batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]
